In [2]:
from pymatgen.ext.matproj import MPRester
from pymatgen.core import Structure
from pymatgen.io.vasp.outputs import Oszicar, Outcar, Vasprun
from pymatgen.core.surface import SlabGenerator
from pymatgen.analysis.surface_analysis import get_slab_regions
import slab_functions as sf
import numpy as np
import crystal_toolkit

In [3]:
#Import the relaxed catio3 structure

contcar = Structure.from_file("CONTCAR")
print(contcar)
contcar.volume

Full Formula (Ca4 Ti4 O12)
Reduced Formula: CaTiO3
abc   :   5.409826   5.516706   7.700753
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (20)
  #  SP            a         b         c
---  ----  ---------  --------  --------
  0  Ca     0.509596  0.544671  0.75
  1  Ca     0.490404  0.455329  0.25
  2  Ca     0.990404  0.044671  0.75
  3  Ca     0.009596  0.955329  0.25
  4  Ti    -0         0.5       0.5
  5  Ti     0         0.5       0
  6  Ti     0.5       0         0.5
  7  Ti     0.5       0         0
  8  O      0.079636  0.479698  0.75
  9  O      0.920364  0.520302  0.25
 10  O      0.420364  0.979698  0.75
 11  O      0.579636  0.020302  0.25
 12  O      0.791502  0.790711  0.958162
 13  O      0.208498  0.209289  0.458162
 14  O      0.208498  0.209289  0.041838
 15  O      0.791502  0.790711  0.541838
 16  O      0.708498  0.290711  0.958162
 17  O      0.291502  0.709289  0.458162
 18  O      0.291502  0.709289  0.041838
 19  O    

/Users/eyob/Library/Python/3.9/lib/python/site-packages/pymatgen/core/structure.py:3083: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="rt", errors="replace") as file:


229.82451656102154

# System characteristics

Number of atoms in bulk system: 20
- 4 Ca
- 4 Ti
- 12 O

Energy of the relaxed bulk system: -160.02 eV

Energy of the relaxed bulk system/atom: -8.001 eV

Lattice parameters: a < b < c

Cell volume: 229.82 A ^ 3

In [4]:
# Import the catio3 structure from Materials Project (MP)

mpr = MPRester("GVPDJzGk7QCVi80Gnm6QCYrKAlJiTtLf")
mp_id = "mp-4019"   # Material id for orthorombic CaTiO3
cto_mp_struct = mpr.get_structure_by_material_id(mp_id)
cto_mp_struct.add_oxidation_state_by_element({"Ca":2, "Ti":4, "O":-2})
print(cto_mp_struct)

Full Formula (Ca4 Ti4 O12)
Reduced Formula: CaTiO3
abc   :   5.372049   5.462796   7.636360
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (20)
  #  SP           a         b          c    magmom
---  ----  --------  --------  ---------  --------
  0  Ca2+  0.509067  0.542398   0.75             0
  1  Ca2+  0.490933  0.457602   0.25             0
  2  Ca2+  0.990933  0.042398   0.75             0
  3  Ca2+  0.009067  0.957602   0.25             0
  4  Ti4+  0         0.5        0.5             -0
  5  Ti4+  0         0.5       -0               -0
  6  Ti4+  0.5       0          0.5             -0
  7  Ti4+  0.5       0         -0               -0
  8  O2-   0.076781  0.481328   0.75             0
  9  O2-   0.923219  0.518672   0.25             0
 10  O2-   0.423219  0.981328   0.75             0
 11  O2-   0.576781  0.018672   0.25             0
 12  O2-   0.790294  0.789614   0.959841        -0
 13  O2-   0.209706  0.210386   0.459841        -0

In [5]:
# Difference in lattice parameters between original structure and structure after relaxtion
diff_in_lattice_param = np.array([contcar.lattice.a - cto_mp_struct.lattice.a, contcar.lattice.b - cto_mp_struct.lattice.b, contcar.lattice.c - cto_mp_struct.lattice.c])
mp_lattice_param  = np.array([cto_mp_struct.lattice.a, cto_mp_struct.lattice.b, cto_mp_struct.lattice.c])
percent_error = (diff_in_lattice_param / mp_lattice_param) * 100
print(f"Percent error in a: {percent_error[0]:.2}0 %")
print(f"Percent error in b: {percent_error[1]:.2} %")
print(f"Percent error in c: {percent_error[2]:.2} %")
print("Percent error in alpha = beta = gamma: 0 %")

Percent error in a: 0.70 %
Percent error in b: 0.99 %
Percent error in c: 0.84 %
Percent error in alpha = beta = gamma: 0 %


In [6]:
contcar.matches(cto_mp_struct)

True

In [7]:
# 001 Miller index
slabgen_001 = SlabGenerator(contcar, (0,0,1), 10, 10) # Structure, Miller index, minimum slab size, minimum vacuum size
slabs_001 = slabgen_001.get_slabs(symmetrize=True) # Generate surfaces with a Miller index of (0, 0, 1)
print(len(slabs_001))

print(sf.match_slabs(slabs_001))
print(sf.polar_slabs(slabs_001))
print(sf.symmetric_slabs(slabs_001))

/Users/eyob/Library/Python/3.9/lib/python/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['wyckoffs']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/Users/eyob/Library/Python/3.9/lib/python/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['equivalent_atoms']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(


4
False
False
False


In [8]:
# 010 Miller index
slabgen_010 = SlabGenerator(contcar, (0,1,0), 10, 10) # Structure, Miller index, minimum slab size, minimum vacuum size
slabs_010 = slabgen_010.get_slabs(symmetrize=True) # Generate surfaces with a Miller index of (0, 1, 0)
print(len(slabs_010))

print(sf.match_slabs(slabs_010))
print(sf.polar_slabs(slabs_010))
print(sf.symmetric_slabs(slabs_010))

7
False
False
False


In [9]:
# 100 Miller index
slabgen_100 = SlabGenerator(contcar, (1,0,0), 10, 10) # Structure, Miller index, minimum slab size, minimum vacuum size
slabs_100 = slabgen_100.get_slabs(symmetrize=True) # Generate surfaces with a Miller index of (1, 0, 0)
print(len(slabs_100))

print(sf.match_slabs(slabs_100))
print(sf.polar_slabs(slabs_100))
print(sf.symmetric_slabs(slabs_100))

5
False
False
False


In [10]:
# 110 Miller Index
slabgen_110 = SlabGenerator(contcar, (1,1,0), 10, 10) # Structure, Miller index, minimum slab size, minimum vacuum size
slabs_110 = slabgen_110.get_slabs(symmetrize=True) # Generate surfaces with a Miller index of (1, 1, 0)
print(len(slabs_110))

print(sf.match_slabs(slabs_110))
print(sf.polar_slabs(slabs_110))
print(sf.symmetric_slabs(slabs_110))

6
False
False
False


In [11]:
# 101 Miller Index
slabgen_101 = SlabGenerator(contcar, (1,0,1), 10, 10) # Structure, Miller index, minimum slab size, minimum vacuum size
slabs_101 = slabgen_101.get_slabs(symmetrize=True) # Generate surfaces with a Miller index of (1, 0, 1)
print(len(slabs_101))

print(sf.match_slabs(slabs_101)) # slabs_101[2] and slabs_101[3] match
print(sf.polar_slabs(slabs_101))
print(sf.symmetric_slabs(slabs_101))

8
[(2, 3), (3, 2)]
False
False


In [12]:
# 011 Miller Index
slabgen_011 = SlabGenerator(contcar, (0,1,1), 10, 10) # Structure, Miller index, minimum slab size, minimum vacuum size
slabs_011 = slabgen_011.get_slabs(symmetrize=True) # Generate surfaces with a Miller index of (0, 1, 1)
print(len(slabs_011))

print(sf.match_slabs(slabs_011))
print(sf.polar_slabs(slabs_011))
print(sf.symmetric_slabs(slabs_011))

6
False
False
False


In [13]:
# 111 Miller Index
slabgen_111 = SlabGenerator(contcar, (1,1,1), 10, 10) # Structure, Miller index, minimum slab size, minimum vacuum size
slabs_111 = slabgen_111.get_slabs(symmetrize=True) # Generate surfaces with a Miller index of (1, 1, 1)
print(len(slabs_111))

print(sf.match_slabs(slabs_111)) # slabs_111[0] and slabs_111[1] match, slabs_111[4] and slabs_111[5] match, slabs_111[6] and slabs_111[7] match, slabs_111[10] and slabs_111[11] match
print(sf.polar_slabs(slabs_111))
print(sf.symmetric_slabs(slabs_111))

12
[(0, 1), (1, 0), (4, 5), (5, 4), (6, 7), (7, 6), (10, 11), (11, 10)]
False
False


48 slabs generated in total

0 polar slabs generated

0 non-symmetric slabs

2/8 surfaces generated by the 101 Miller index are "matching"

8/12 surfaces generated by the 111 Miller index are "matching"

43 non-matching slabs generated in total

# Slab generation to do convergence tests

In [14]:
# 001 Miller index
slabgen_001_converge = []
for i in range(1, 35):
    slabgen_001_i = SlabGenerator(contcar, (0,0,1), 10, i) # Structure, Miller index, minimum slab size, minimum vacuum size
    slabs_001_i = slabgen_001_i.get_slabs(symmetrize=True) # Generate surfaces with a Miller index of (0, 0, 1)
    slabgen_001_converge.append(slabs_001_i)


In [15]:
# Access the first termination of the 001 Miller index for each vacuum size
# Don't know if the first termination for each vacuum size represents the same system where vacuum
# size is the only difference, but for now we assume.

first_term = []
for vac_size in slabgen_001_converge:
    first_term.append(vac_size[0])

In [16]:
c_param = 0
for i, vac_size in enumerate(first_term):
    if vac_size.lattice.c != c_param:
        c_param = vac_size.lattice.c
        vac_size.to(filename=f"001_facet_01_term_vac_{vac_size.lattice.c:.2f}.POSCAR")

/Users/eyob/Library/Python/3.9/lib/python/site-packages/pymatgen/io/vasp/inputs.py:624: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="wt") as file:


In [17]:
def surface_energy(area, e_slab, e_bulk, n_slab):
    coeff = 1 / 2 * area
    return coeff * (e_slab - n_slab * e_bulk)

# Test vacuum convergence script

In [18]:
sf.vac_converge(contcar, (0,0,1), 35)

In [19]:
sf.vac_converge(contcar, (0,1,0), 35)

/Users/eyob/Library/Python/3.9/lib/python/site-packages/pymatgen/io/vasp/inputs.py:624: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="wt") as file:


In [20]:
sf.vac_converge(contcar, (1,0,0), 35)

In [21]:
sf.vac_converge(contcar, (1,1,0), 35)

In [22]:
sf.vac_converge(contcar, (1,0,1), 35)

In [23]:
sf.vac_converge(contcar, (0,1,1), 35)

In [24]:
sf.vac_converge(contcar, (1,1,1), 35)